http://www.fenris.org/2016/03/07/index-html

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout, BatchNormalization, Embedding, Reshape, merge, Merge
from keras.callbacks import Callback, EarlyStopping

Using TensorFlow backend.


In [130]:
uids = np.array([1, 1, 2, 3, 4, 4])
mids = np.array([1, 3, 2, 2, 3, 4])
rates = np.array([3, 4, 5, 2, 1, 3])
num_user = 4
num_POI = 4
vec_dim = 20

In [131]:
userModel = Sequential()
userModel.add(Embedding(num_user+1, output_dim=vec_dim, input_length=1))
userModel.add(Reshape((vec_dim,)))

In [132]:
userModel.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_16 (Embedding)     (None, 1, 20)             100       
_________________________________________________________________
reshape_24 (Reshape)         (None, 20)                0         
Total params: 100
Trainable params: 100
Non-trainable params: 0
_________________________________________________________________


In [133]:
poiModel = Sequential()
poiModel.add(Embedding(num_POI+1, output_dim=vec_dim, input_length=1))
poiModel.add(Reshape((vec_dim,)))

In [134]:
poiModel.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_17 (Embedding)     (None, 1, 20)             100       
_________________________________________________________________
reshape_25 (Reshape)         (None, 20)                0         
Total params: 100
Trainable params: 100
Non-trainable params: 0
_________________________________________________________________


We’ve just created a left embedding layer that creates a Users by Latent Factors matrix and a right embedding layer that creates a Movies by Latent Factors matrix.  

When the input to these is a user id and a movie id, then they return the latent factor vectors for the user and the movie, respectively.  

In [135]:
cfModel = Sequential()
merge = Merge([userModel, poiModel], mode='dot')
cfModel.add(merge)

/Users/bgg/training_course/lib/python2.7/site-packages/ipykernel/__main__.py:2: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  from ipykernel import kernelapp as app


In [136]:
cfModel.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
merge_31 (Merge)             (None, 1)                 0         
Total params: 200
Trainable params: 200
Non-trainable params: 0
_________________________________________________________________


The Merge layer then takes the dot product of these two things to return rating. 

We compile the model using MSE as the loss function and the AdaMax learning algorithm (which is superior to Sparse Gradient Descent).  

Our callbacks monitor the validation loss and we save the model weights each time the validation loss has improved.

In [137]:
callbacks = [EarlyStopping('val_loss', patience=2), ModelCheckpoint('movie_weights.h5', save_best_only=True)]

In [141]:
cfModel.compile(loss='mse', optimizer='adam')

In [142]:
uids = uids.reshape((len(uids), 1))
mids = mids.reshape((len(mids),1))
rates = rates.reshape((len(rates),1))

In [143]:
cfModel.fit([uids, mids], rates, nb_epoch=50, validation_split=.1, callbacks=callbacks, verbose=2)

Train on 5 samples, validate on 1 samples
Epoch 1/50
0s - loss: 10.9782 - val_loss: 9.0189
Epoch 2/50
0s - loss: 10.9731 - val_loss: 9.0199
Epoch 3/50
0s - loss: 10.9680 - val_loss: 9.0209
Epoch 4/50
0s - loss: 10.9628 - val_loss: 9.0219


In [144]:
def durConv(n):
        m = n**(0.2) / (28800**0.2)
        return max(0, min(1, m))

In [146]:
durConv(100)

0.3221970074886271

In [5]:
kk = pd.read_csv('data/kktv/data-001.csv')
kk.head(30)

,user_id,device_id,session_id,title_id,event_time,played_duration,action_trigger,platform,episode_number,series_total_episodes_count,internet_connection_type,is_trailer
0,0,0,0,0,2017-06-08 14:54:30.325,2,seek,iOS,12,16,wifi,False
1,0,0,0,0,2017-06-08 14:54:34.857,2,seek,iOS,12,16,wifi,False
2,0,0,0,0,2017-06-08 14:54:36.730,1,seek,iOS,12,16,wifi,False
3,0,0,0,0,2017-06-08 14:54:38.976,1,seek,iOS,12,16,wifi,False
4,0,0,0,0,2017-06-08 14:54:40.373,1,seek,iOS,12,16,wifi,False


In [12]:
kk[kk.user_id==4]

,user_id,device_id,session_id,title_id,event_time,played_duration,action_trigger,platform,episode_number,series_total_episodes_count,internet_connection_type,is_trailer
4142,4,6,41,17,2017-07-06 11:02:20.502,5,pause,Android,2,18,wifi,False
4143,4,6,41,17,2017-07-06 11:02:27.715,3,pause,Android,2,18,wifi,False
4144,4,6,41,17,2017-07-06 11:02:32.742,2,seek,Android,2,18,wifi,False
4145,4,6,41,17,2017-07-06 11:02:39.411,4,seek,Android,2,18,wifi,False
4146,4,6,41,17,2017-07-06 11:02:42.530,1,seek,Android,2,18,wifi,False
4147,4,6,41,17,2017-07-06 11:02:48.742,4,pause,Android,2,18,wifi,False
4148,4,6,41,17,2017-07-06 11:03:02.393,10,seek,Android,2,18,wifi,False
4149,4,6,41,17,2017-07-06 11:03:04.685,1,seek,Android,2,18,wifi,False
4150,4,6,41,17,2017-07-06 11:03:07.877,2,seek,Android,2,18,wifi,False
4151,4,6,41,17,2017-07-06 11:03:17.444,8,pause,Android,2,18,wifi,False
